**Importing Libraries**

In [43]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping

In [44]:
number_of_transactions = 2

In [45]:
data = pd.read_csv('../input/amex-default-prediction/train_data.csv', nrows=200000).groupby('customer_ID').tail(number_of_transactions).set_index('customer_ID', drop=True).sort_index()
labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv').set_index('customer_ID', drop=True).sort_index()

**Merging the Training data with Labels**

In [46]:
df = pd.merge(data, labels, left_index=True, right_index=True)  

In [47]:
df.head()

,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2018-02-21,0.937349,0.002425,0.019837,1.008307,0.000607,0.102985,0.006174,0.007454,NaN,...,NaN,NaN,0.001563,0.001976,0.002569,NaN,0.001098,0.006346,0.004204,0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2018-03-13,0.934745,0.009119,0.009382,1.007647,0.006104,0.135021,0.001604,0.007174,NaN,...,NaN,NaN,0.007186,0.004234,0.005086,NaN,0.005810,0.002970,0.008533,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,2018-02-06,0.878856,0.536290,0.034558,1.005419,0.007248,0.159486,0.006274,0.008733,NaN,...,NaN,NaN,0.006658,0.004627,0.004820,NaN,0.008272,0.002528,0.003584,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,2018-03-25,0.880519,0.178126,0.034684,1.004028,0.006911,0.165509,0.005552,0.005068,NaN,...,NaN,NaN,0.002980,0.007479,0.007870,NaN,0.003284,0.003169,0.008514,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,2018-02-09,0.866342,0.001526,0.008318,0.818953,0.006756,NaN,0.009511,0.004373,NaN,...,NaN,NaN,0.008112,0.009543,0.003328,NaN,0.004781,0.007325,0.005606,0


**Descriptive Statistics**

In [48]:
df.describe()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
count,32759.000000,3.297800e+04,32978.000000,32975.000000,3.297800e+04,26809.000000,3.297500e+04,3.297500e+04,4422.000000,23348.000000,...,1231.000000,1231.000000,3.272900e+04,32972.000000,3.272900e+04,5767.000000,3.272900e+04,3.297200e+04,3.272900e+04,32978.000000
mean,0.632882,1.969083e-01,0.140342,0.585763,1.076098e-01,0.231808,9.378779e-02,1.556723e-01,0.178078,0.165073,...,0.018771,0.145233,1.812338e-01,0.028439,1.666252e-01,0.394600,1.811465e-01,5.212858e-02,6.346008e-02,0.264085
std,0.266624,3.694746e-01,0.231423,0.413427,2.781923e-01,0.204968,2.873838e-01,2.537062e-01,0.243863,0.228746,...,0.116713,0.253666,3.810453e-01,0.151332,3.501898e-01,0.245088,3.809258e-01,1.818579e-01,1.948796e-01,0.440852
min,-0.355230,7.301857e-07,-0.309330,0.000006,6.440720e-07,-0.240078,6.066678e-07,1.323113e-07,-0.000219,0.000001,...,0.000007,0.000014,3.833695e-07,0.000001,1.323148e-07,-0.009400,1.653580e-08,7.164186e-07,4.609489e-08,0.000000
25%,0.447735,4.974273e-03,0.009007,0.068823,3.001480e-03,0.128562,3.014322e-03,5.520861e-03,0.033735,0.043012,...,0.002437,0.003324,3.060873e-03,0.002539,3.017133e-03,0.193622,3.046739e-03,2.706759e-03,3.001234e-03,0.000000
50%,0.678158,9.976387e-03,0.034422,0.813246,5.952563e-03,0.165469,5.992414e-03,1.118144e-02,0.115234,0.091015,...,0.004973,0.006791,6.099273e-03,0.005125,6.023944e-03,0.380992,6.103070e-03,5.460053e-03,6.053148e-03,0.000000
75%,0.859826,2.710732e-01,0.158221,1.001860,8.901228e-03,0.266837,8.974754e-03,2.178614e-01,0.235525,0.196589,...,0.007673,0.500239,9.095845e-03,0.007666,9.105157e-03,0.581989,9.120776e-03,8.242189e-03,9.087650e-03,1.000000
max,1.009943,5.036047e+00,1.324053,1.009999,2.507711e+00,2.888382,6.798167e+00,1.625262e+00,3.249918,5.794954,...,1.009288,1.509486,1.010000e+00,1.009992,1.174753e+00,1.751388,1.009993e+00,1.342748e+00,4.187805e+00,1.000000


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32978 entries, 0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a to 09572fafe01b8bbb560809da84c7d1c8d9e79eb7287bc5a2d1e1a4a474ab5382
Columns: 190 entries, S_2 to target
dtypes: float64(185), int64(2), object(3)
memory usage: 48.1+ MB


In [50]:
df.isnull().sum()

S_2           0
P_2         219
D_39          0
B_1           0
B_2           3
          ...  
D_142     27211
D_143       249
D_144         6
D_145       249
target        0
Length: 190, dtype: int64

# Pre-Processing

**Dropping the Transaction Dates**

In [51]:
drop_cols = ['S_2'] 
df.drop(drop_cols, inplace=True, axis=1)

**Creating Training Labels and Data**

In [52]:
df_dummy= pd.get_dummies(df)

In [53]:
y = df_dummy['target']
X = df_dummy.drop('target', axis=1)

**Handling missing values**

Missing values are imputed with the respective column mean.

In [54]:
col_names = X.columns
imputer = SimpleImputer()
X = pd.DataFrame(imputer.fit_transform(X))  
X.columns = col_names

**Standardization**

In [55]:
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns) 

**Splitting into training and validation sets**

In [56]:
#train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1, shuffle=True, stratify=y, test_size=0.2)
X_train, X_test , y_train , y_test = train_test_split(X, y, random_state = 1, shuffle=True, stratify=y, test_size=0.2)

Memory cleanup to prevent out of memory error.

In [57]:
del data, labels, X, y
gc.collect()

192

# Model Building

**Logistic Regression**

In [58]:
from sklearn.metrics import classification_report,accuracy_score
from sklearn import metrics

In [59]:
from sklearn.linear_model import LogisticRegression

In [60]:
clf_lr = LogisticRegression(n_jobs=1, C=1e5)
clf_lr.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(C=100000.0, n_jobs=1)

In [61]:
%%time
y_pred_val_lr = clf_lr.predict(X_test)
print('Accuracy on Validation set :',accuracy_score(y_test, y_pred_val_lr))
print("\n")
print(classification_report(y_test, y_pred_val_lr))

Accuracy on Validation set : 0.8946331109763493


              precision    recall  f1-score   support

           0       0.92      0.94      0.93      4854
           1       0.81      0.78      0.80      1742

    accuracy                           0.89      6596
   macro avg       0.87      0.86      0.86      6596
weighted avg       0.89      0.89      0.89      6596

CPU times: user 95 ms, sys: 94.7 ms, total: 190 ms
Wall time: 49.3 ms


In [62]:
from lightgbm import LGBMClassifier

In [63]:
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)

LGBMClassifier()

In [64]:
%%time
y_pred_val_lgbm = lgbm.predict(X_test)


CPU times: user 89.5 ms, sys: 3.99 ms, total: 93.5 ms
Wall time: 35.1 ms


In [65]:
from catboost import CatBoostClassifier

In [66]:
clf = CatBoostClassifier(iterations = 3000, random_state = 42, nan_mode ='Min',task_type ="GPU")
clf.fit(X_train, y_train)
preds = clf.predict(X_test)

CatBoostError: catboost/cuda/cuda_lib/cuda_manager.cpp:201: Condition violated: `State == nullptr'

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
stand_scaler = StandardScaler()
x_train_norm = stand_scaler.fit_transform(X_train)
x_test_norm = stand_scaler.fit_transform(X_test)



param_grid = { 
    'C': [6,7,8],
}
clf = LinearSVC(class_weight='balanced', dual=False, penalty = "l1", random_state = 42)
svc_random = GridSearchCV(estimator = clf, param_grid = param_grid, cv=2, verbose=0, n_jobs = -1)
svc_random.fit(x_train_norm,y_train)
svc_model = svc_random.best_estimator_


y_pred = svc_model.predict(x_test_norm)




In [ ]:
df_subm = pd.read_csv('../input/amex-default-prediction/test_data.csv')

In [ ]:
df_subm["prediction"] = y_pred_val_lgbm

In [ ]:
result = df_subm[['customer_ID','prediction']]

In [ ]:
import base64
from IPython.display import HTML
def download_csv(df, title = "Download CSV file", filename = "output_180474E.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [ ]:
download_csv(result)